In [14]:
try:
  from fredapi import Fred
except:
  !pip install fredapi > /dev/null
  from fredapi import Fred

In [62]:
try:
  import yfinance as yf
except:
  !pip install yfinance > /dev/null
  import yfinance as yf

In [15]:
from google.colab import userdata
import pandas as pd

In [16]:
fred = Fred(api_key=userdata.get('fredkey'))

# Question 1
Average GDP growth in 2023

In [30]:
# Get data
data = pd.DataFrame(fred.get_series('GDPC1')).rename(columns={0:"gdp"})

In [31]:
# Filter data of interest, from 2022 onwards
data = data[data.index >= pd.to_datetime("2022-01-01")]

In [36]:
# Calculate lagged values and growth rates
data["lag_gdp"] = data["gdp"].shift(4)
data["yoy_growth"] = data.apply(lambda row: ((row["gdp"]/row["lag_gdp"])-1)*100, axis=1)

In [39]:
# Restrict the range to 20203 to calculate average growth rate
average_growth = data[(data.index >= pd.to_datetime("2023-01-01")) &
 (data.index <= pd.to_datetime("2023-12-31"))]["yoy_growth"].mean()

In [40]:
round(average_growth, 1)

2.5

# Question 2

Minimum inverse "Treasury Yield" since 2000

In [57]:
# Get data, rename index for easier merge
dgs2 = pd.DataFrame(fred.get_series('DGS2')).rename(columns={0:"dgs2"}).rename_axis("date", axis="index")
dgs10 = pd.DataFrame(fred.get_series('DGS10')).rename(columns={0:"dgs10"}).rename_axis("date", axis="index")

In [58]:
dgs_df = dgs2.merge(dgs10, on="date")

In [59]:
# Filter data of interest, from 2000 onwards
dgs_df = dgs_df[dgs_df.index >= pd.to_datetime("2000-01-01")]

In [60]:
# Calculate inverse treasury yield and find minimum
dgs_df["inv_ty"] = dgs_df.apply(lambda row: row["dgs10"]-row["dgs2"], axis=1)

In [61]:
round(dgs_df["inv_ty"].min(),1)

-1.1

# Question 3

Which Index is better recently?

In [69]:
indexes_df = yf.download("^GSPC ^MXX", start="2019-04-09", end="2024-04-10") # End is excluded, need to add one day

[*********************100%%**********************]  2 of 2 completed


In [81]:
sp500_growth = ((
    (indexes_df["Close"]["^GSPC"].tail(1).values /
     indexes_df["Close"]["^GSPC"].head(1).values) - 1)
     * 100
)

In [82]:
sp500_growth

array([81.01279427])

In [80]:
ipc_growth = ((
    (indexes_df["Close"]["^MXX"].tail(1).values /
     indexes_df["Close"]["^MXX"].head(1).values) - 1)
     * 100
)
ipc_growth

array([27.50624913])

# Question 4

52-week range ratio for selected stocks

In [87]:
# Create the list of stocks
stocks_list = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]

In [88]:
stocks_df = yf.download(" ".join(stocks_list), start="2023-01-01", end="2024-01-01") # End is excluded, need to add one day

[*********************100%%**********************]  6 of 6 completed


In [94]:
# find max and min adj close for each stock and save in a dict
results_list = []

for stock in stocks_list:
  min_close = stocks_df["Adj Close"][stock].min()
  max_close = stocks_df["Adj Close"][stock].max()
  results_list.append({
      "stock": stock,
      "min": min_close,
      "max":max_close,
      "range_ratio": (max_close-min_close)/max_close})

In [97]:
results_list.sort(key=lambda x: x["range_ratio"])

In [99]:
results_list[-1]

{'stock': 'MSFT',
 'min': 219.94937133789062,
 'max': 381.9936218261719,
 'range_ratio': 0.4242066914981641}

# Question 5

Dividend Yield

In [136]:
results_dividend = []
for stock in stocks_list:
  handle = yf.Ticker(stock)
  # Get dividendds
  div_df = pd.DataFrame(handle.dividends)
  # Need to add utc=True for comparison with index
  div_df = div_df[(div_df.index >= pd.to_datetime("2023-01-01", utc=True)) &
                  (div_df.index <= pd.to_datetime("2024-01-01", utc=True))]
  dividend_total = div_df.Dividends.sum()
  stock_data = yf.download(stock, start="2023-12-20", end="2024-01-01")
  last_close = stock_data["Adj Close"].tail(1).values[0]
  results_dividend.append({
      "stock": stock,
      "dividend_total": dividend_total,
      "last_close": last_close,
      "dividend_yield":round((dividend_total/last_close)*100,1)
  })

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [137]:
results_dividend.sort(key=lambda x: x["dividend_yield"])

results_dividend[-1]

{'stock': '2222.SR',
 'dividend_total': 0.9107640000000001,
 'last_close': 32.82804870605469,
 'dividend_yield': 2.8}

# Question 6

Explore additional metrics

In [139]:
saudi = yf.Ticker("2222.SR")

In [140]:
saudi.income_stmt

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,0.0,0.0,0.0,5800000.0
Tax Rate For Calcs,0.2,0.2,0.2,0.2
Normalized EBITDA,993293000000.0,1253462000000.0,867100000000.0,459167000000.0
Total Unusual Items,NaN,NaN,13000000.0,29000000.0
Total Unusual Items Excluding Goodwill,NaN,NaN,13000000.0,29000000.0
Net Income From Continuing Operation Net Minority Interest,452753000000.0,597215000000.0,395203000000.0,184926000000.0
Reconciled Depreciation,97040000000.0,91618000000.0,85521000000.0,76208000000.0
Reconciled Cost Of Revenue,664788000000.0,684220000000.0,659186000000.0,421638000000.0
EBITDA,993293000000.0,1253462000000.0,867100000000.0,459196000000.0
EBIT,896253000000.0,1161844000000.0,781579000000.0,382988000000.0


In [141]:
saudi.quarterly_income_stmt

,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.2,0.2,0.2,0.2
Normalized EBITDA,231952000000.0,264676000000.0,242279000000.0,254386000000.0
Net Income From Continuing Operation Net Minority Interest,102867000000.0,123534000000.0,108881000000.0,117471000000.0
Reconciled Depreciation,28018000000.0,24355000000.0,22692000000.0,21975000000.0
Reconciled Cost Of Revenue,3402000000.0,223685000000.0,217982000000.0,219719000000.0
EBITDA,231952000000.0,264676000000.0,242279000000.0,254386000000.0
EBIT,203934000000.0,240321000000.0,219587000000.0,232411000000.0
Net Interest Income,1804000000.0,4941000000.0,5668000000.0,7791000000.0
Interest Expense,905000000.0,1948000000.0,2156000000.0,3177000000.0


In [142]:
saudi.cashflow

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Free Cash Flow,379506000000.0,556991000000.0,402956000000.0,184267000000.0
Repurchase Of Capital Stock,NaN,NaN,NaN,0.0
Repayment Of Debt,-134495000000.0,-132514000000.0,-73563000000.0,-42125000000.0
Issuance Of Debt,32057000000.0,9082000000.0,42213000000.0,87520000000.0
Issuance Of Capital Stock,662000000.0,550000000.0,384000000.0,NaN
Capital Expenditure,-158308000000.0,-141161000000.0,-119645000000.0,-101030000000.0
End Cash Position,198973000000.0,226047000000.0,299579000000.0,207232000000.0
Beginning Cash Position,226047000000.0,299579000000.0,207232000000.0,177706000000.0
Changes In Cash,-27074000000.0,-73532000000.0,92347000000.0,29526000000.0
Financing Cash Flow,-510869000000.0,-382675000000.0,-294513000000.0,-234872000000.0


In [143]:
saudi.quarterly_cashflow

,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Free Cash Flow,100540000000.0,76280000000.0,86836000000.0,115850000000.0
Repayment Of Debt,-12001000000.0,-1585000000.0,-61058000000.0,-59851000000.0
Issuance Of Debt,11854000000.0,2406000000.0,2089000000.0,15708000000.0
Issuance Of Capital Stock,165000000.0,177000000.0,165000000.0,155000000.0
Capital Expenditure,-44918000000.0,-41354000000.0,-39239000000.0,-32797000000.0
End Cash Position,198973000000.0,191022000000.0,248891000000.0,340674000000.0
Beginning Cash Position,191022000000.0,248891000000.0,340674000000.0,226047000000.0
Changes In Cash,7951000000.0,-57869000000.0,-91783000000.0,114627000000.0
Financing Cash Flow,-123132000000.0,-117819000000.0,-145574000000.0,-124344000000.0
Interest Paid Cff,-4698000000.0,-2575000000.0,-4875000000.0,-2204000000.0
